## Data606 - Capstone Project
```
Group H
Malav Patel, Kent Butler
Prof. Unal Sokaglu
```

This project is about performing time-series analysis on climate data analysis data.



# Research

### References

Some explanations of earth sciences statistics:
https://pjbartlein.github.io/REarthSysSci/ltms-and-anomalies.html

NOAA PSL NCEP-NCAR datasets:  https://psl.noaa.gov/data/gridded/data.ncep.reanalysis.html

NOAA PSL, other recognized data sources directory: https://psl.noaa.gov/data/help/othersources/

Global environmental policy timeline, https://www.boell.de/en/2022/05/28/international-environmental-policy-timeline

OECD convergence of policy, climate,and economy: https://www.oecd.org/

NASA climate time machine: https://climate.nasa.gov/interactives/climate-time-machine

### Factoids

* All of the plastic waste produced in the world in 2019 alone weighs as much as 35,000 Eiffel Towers – 353 million tons  - [*Organization for Economic Cooperation and Development (OECD)*](https://www.boell.de/en/2022/05/28/international-environmental-policy-timeline)



## Application Parameters

Note: algorithm tuning is done with declaration of the model.

In [ ]:
import pandas as pd
from datetime import datetime as dt
import datetime

In [ ]:
debug = False

DRIVE_PATH = "/content/drive/MyDrive/data606"

# Set the location of this script in GDrive
SCRIPT_PATH = DRIVE_PATH + "/src/"

# Model to use
MODEL_NAME = "LSTMv2"

# Journal file
JOURNAL_LOG = SCRIPT_PATH + "cv-results.csv"

# Number of samples to work with - will be split  into train/test
SAMPLE_SIZE = 5000

# Device to run on
run_on_device =  'cpu' # 'cuda'


**Configure Predictions**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd $SCRIPT_PATH

# Data Load

In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
plt.rcParams["figure.figsize"] = (10,6)
import warnings

In [ ]:
warnings.filterwarnings('ignore')
plt.style.use('seaborn')
%matplotlib inline

---

**Initial Data Load**

---

In [ ]:
# Load util class
%run -i "./Dataset_Merger.ipynb"

In [ ]:
class ModelExecutor():

  def __init__(self, data_path, start_date, end_date, input_window, label_window, test_ratio, val_ratio, num_epochs, model_name, debug=False):
    self.debug = debug
    self.data_path = data_path
    self.start_date = start_date
    self.end_date = end_date
    self.INPUT_WINDOW = input_window
    self.LABEL_WINDOW = label_window
    self.TEST_RATIO = test_ratio
    self.val_ratio = val_ratio
    self.num_epochs = num_epochs
    self.MODEL_NAME = model_name


  def load_initial_dataset(self, ds_name, feature_map, date_map=None, date_col=None):
    # Declare a merger compatible with our source data and our target dataset we want to merge into
    self.merger = Dataset_Merger(data_path=self.data_path,
                            start_date=self.start_date, end_date=self.end_date,
                            debug=self.debug)

    # Start by merging initial dataset
    if (date_map is not None):
      self.df_merge = self.merger.merge_dataset(ds_name, feature_map, date_map=date_map)
    elif (date_col is not None):
      self.df_merge = self.merger.merge_dataset(ds_name, feature_map, date_col=date_col)

    print(assess_na(self.df_merge))


  def load_datasets(self, ds_list):
    for dataset in ds_list:
      if ('date_map' in dataset):
        self.df_merge = self.merger.merge_dataset(dataset['filename'],
                                        feature_map=dataset['feature_map'],
                                        df_aggr=self.df_merge,
                                        date_map=dataset['date_map'])
      else:
        self.df_merge = self.merger.merge_dataset(dataset['filename'],
                                    feature_map=dataset['feature_map'],
                                    df_aggr=self.df_merge,
                                    date_col=dataset['date_col'])

      if (self.debug):
        print(df_merge)
        print(assess_na(df_merge))


  def print_correlations(self):
    # Assess correlations between all data columns
    df_corr = df_merge.corr()

    # Identify the columns which have medium to strong correlation with target
    df_corr_cols = df_corr[df_corr[TARGET_LABEL] > 0.5]

    # Drop the target from the correlation results in case we want to use this reduced set
    #    in place of the full set
    df_corr_cols = df_corr_cols.drop(columns=[])

    # Extract just the column names
    corr_cols = df_corr_cols.index.values

    if debug:
      print(corr_cols)

    # Add labels
    labels = np.where(np.abs(df_corr) > 0.75, 'S',
                      np.where(np.abs(df_corr) > 0.5, 'M',
                              np.where(np.abs(df_corr) > 0.25, 'W', '')))
    # Plot the matrix
    plt.figure(figsize=(8,6))
    sns.heatmap(df_corr, mask=np.eye(len(df_corr)), square=True,
                center=0, annot=labels, fmt = '', linewidths = .5,
                cmap='vlag', cbar_kws={'shrink':0.8});
    plt.title('Heatmap of correlation among variables', fontsize=20)


  def process(self):

    from sklearn.preprocessing import StandardScaler, MinMaxScaler
    from sklearn import preprocessing
    from sklearn.compose import ColumnTransformer
    from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler, RobustScaler, PowerTransformer,  QuantileTransformer, Normalizer
    from sklearn.impute import SimpleImputer
    from sklearn.pipeline import Pipeline
    import tensorflow as tf

    NUM_FEATURES = len(df_merge.columns)

    # Capture stats on number of non-metadata columns - so, exclude some stuff
    #NET_NUM_FEATURES = len(set(df_merge.columns) - set(['year','month','day']))

    # Keep rows aside for post validation?
    TOTAL_ROWS = df_merge.shape[0]
    NUM_VALIDATION = math.floor(TOTAL_ROWS * VALIDATION_RATIO)
    WORKING_ROWS = TOTAL_ROWS - NUM_VALIDATION

    # Split non-validation rows into train/test
    NUM_TEST = math.floor(WORKING_ROWS * TEST_RATIO)
    NUM_TRAIN = WORKING_ROWS - NUM_TEST

    print(f'Num features: {NUM_FEATURES}')
    print(f'Total rows: {TOTAL_ROWS}')
    print(f'Validation rows: {NUM_VALIDATION}')
    print(f'Train rows: {NUM_TRAIN}')
    print(f'Test rows: {NUM_TEST}'

    ## """**Split into Train/Test**"""

    df_train = df_merge.iloc[:NUM_TRAIN, :]
    df_test = df_merge.iloc[NUM_TRAIN:NUM_TRAIN+NUM_VALIDATION, :]
    df_val = df_merge.iloc[NUM_TRAIN+NUM_VALIDATION:, :]

    y_train = df_train[TARGET_LABEL]
    y_test = df_test[TARGET_LABEL]
    y_val = df_val[TARGET_LABEL]

    if debug:
      print(f'df_train: {df_train.shape}')
      print(f'y_train: {y_train.shape}')
      print(f'df_test: {df_test.shape}')
      print(f'y_test: {y_test.shape}')
      print(f'df_val: {df_val.shape}')
      print(f'y_val: {y_val.shape}')


    ## """**Scale data**

    # Doing this **after** the split means that training data doesn't get unfair advantage of looking ahead into the 'future' during test & validation.



    # Create small pipeline for numerical features
    numeric_pipeline = Pipeline(steps = [('impute', SimpleImputer(strategy='mean')),
                                        ('scale', MinMaxScaler())])

    # get names of numerical features
    con_lst = df_train.select_dtypes(include='number').columns.to_list()

    # Transformer for applying Pipelines
    column_transformer = ColumnTransformer(transformers = [('number', numeric_pipeline, con_lst)])

    # Transform data features
    X_train_tx = column_transformer.fit_transform(df_train)
    X_test_tx = column_transformer.transform(df_test)
    X_val_tx = column_transformer.transform(df_val)
    X_train_tx.shape, X_test_tx.shape, X_val_tx.shape

    # Transform labels
    label_scaler = MinMaxScaler()
    y_train_tx = label_scaler.fit_transform(y_train.values.reshape(-1, 1))

    if debug:
      print(f'X_train:scaled: {X_train_tx[0]}')
      print(f'y_train:scaled: {y_train_tx[0]}')


    ## """**Extract X and y**

    # Normally we would do this by explicitly extracting data from our df.
    # However for a time series, we're going to create many small supervised learning sets, so a set of X and y pairs.
    # We should end up with data in a shape ready for batched network input:
    # `batches X time_steps X features`

    if debug:
      print(f'X_train_tx: {X_train_tx.shape}')
      print(f'y_train_tx: {y_train_tx.shape}')

    ## **Modeling**

    # These are the features we are going to be modeling
    COLS = list(df_merge.columns)

    ## """**Slice into Batches**"""

    ds = tf.keras.utils.timeseries_dataset_from_array(
    data=X_train_tx,
    targets=y_train_tx,
    sequence_length=self.input_window)

    print(ds)

    ## """**Prep GPU**"""

    print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

    ##"""**Build model**"""

    # Commented out IPython magic to ensure Python compatibility.
    # Load model class
    # %run -i "./Model_LSTMv2.ipynb"

    model = ModelLSTMv2(window_size=self.INPUT_WINDOW, num_epochs=self.num_epochs, debug=True)

    ##"""**Train model**"""

    #model.train(X, y, NUM_FEATURES)
    model_history = model.train(dataset=ds, num_features=self.num_features)

    # Capture stat
    num_epochs = len(model_history.history['loss'])

    ##"""**Test Predictions**"""

    num_predictions = y_test.shape[0] - self.INPUT_WINDOW

    preds = []
    y_test_vals = []

    for p in range(num_predictions):
      X_pred = X_test_tx[p:p+self.INPUT_WINDOW,:].reshape(-1, self.INPUT_WINDOW, self.NUM_FEATURES)
      y_test_vals.append(y_test[p+self.INPUT_WINDOW])
      # Predict
      pred = model.predict(X_pred)
      # Scale and save
      preds.append(label_scaler.inverse_transform(pred))

    preds = np.array(preds).reshape(num_predictions)

    ##"""**Analyze results**"""

    df_results = pd.DataFrame({'y_test': y_test_vals,
                              'preds': preds},
                              index=[i+1 for i in range(num_predictions)])

    print(df_results)

    df_results.plot()

    plt.figure(figsize=(12,6))
    plt.plot(y_test_vals, 'blue', linewidth=5)
    plt.plot(preds,'r' , linewidth=4)
    plt.legend(('Test','Predicted'))
    plt.show()

    from sklearn.metrics import mean_squared_error, mean_absolute_error
    from sklearn.metrics import mean_absolute_percentage_error
    import csv

    # Calculate MAPE
    m = tf.keras.metrics.MeanAbsolutePercentageError()
    m.update_state(y_test_vals, preds)

    mse = mean_squared_error(y_test_vals, preds)
    mae = mean_absolute_error(y_test_vals, preds)
    mape = m.result().numpy()/100  # adjust Keras output to match scikit

    sk_mape = mean_absolute_percentage_error(y_test_vals, preds)

    print(f'MSE: {mse}')
    print(f'MAE: {mae}')
    print(f'MAPE: {mape}')
    print(f'SKMAPE: {sk_mape}')

    ## """**Journal entry**"""
    with open(JOURNAL_LOG, 'w') as csvfile:
      writer = csv.writer(csvfile)
      #writer.writerow(['DateTime','Model','TargetLabel','NumFeatures','WindowSize','TestPct','NumEpochs','MSE','MAE','MAPE','SKMAPE','Columns'])
      writer.writerow([dt.today().strftime("%Y%m%d-%H%M"),self.MODEL_NAME,self.TARGET_LABEL,self.NUM_FEATURES,self.INPUT_WINDOW,self.TEST_RATIO,num_epochs,mse,mae,mape,sk_mape,COLS])